In [1]:
#
# Working through
# https://github.com/salvadorgarciamunoz/kipet/blob/master/kipet/examples/Ex_7_conc_input_conf_k_aug.py
#

In [2]:
from __future__ import print_function
from kipet.library.TemplateBuilder import *
from kipet.library.PyomoSimulator import *
from kipet.library.ParameterEstimator import *
from kipet.library.VarianceEstimator import *
from kipet.library.data_tools import *
import matplotlib.pyplot as plt
import os
import sys
import inspect
import six
import pandas as pd

In [3]:
filename = '../data_sets/Ex_1_C_data.txt'
D_frame = read_file(filename, directory = '../data_sets')

read dir : /home/paperspace/learn_kipet/kipet_examples/../data_sets/../data_sets/Ex_1_C_data.txt


In [4]:
D_frame.shape

(300, 3)

In [5]:
builder = TemplateBuilder()    
components = {'A':1e-3,'B':0,'C':0}
builder.add_mixture_component(components)
builder.add_parameter('k1',bounds=(0.0,5.0))
builder.add_parameter('k2',bounds=(0.00,2.0))
builder.add_concentration_data(D_frame)

               A         B         C
0.0000  0.001027  0.000000  0.000013
0.0333  0.001015  0.000002  0.000021
0.0667  0.001007  0.000013  0.000018
0.1000  0.000993  0.000025  0.000014
0.1334  0.000982  0.000035  0.000012
...          ...       ...       ...
9.8353  0.000103  0.000043  0.000859
9.8686  0.000104  0.000042  0.000857
9.9020  0.000103  0.000041  0.000859
9.9353  0.000101  0.000043  0.000860
9.9687  0.000103  0.000038  0.000863

[300 rows x 3 columns]


In [6]:
# define explicit system of ODEs
def rule_odes(m,t):
    exprs = dict()
    exprs['A'] = -m.P['k1']*m.Z[t,'A']
    exprs['B'] = m.P['k1']*m.Z[t,'A']-m.P['k2']*m.Z[t,'B']
    exprs['C'] = m.P['k2']*m.Z[t,'B']
    return exprs

builder.set_odes_rule(rule_odes)
opt_model = builder.create_pyomo_model(0.0,10.0)

In [7]:
v_estimator = VarianceEstimator(opt_model)
v_estimator.apply_discretization('dae.collocation',nfe=100,ncp=3,scheme='LAGRANGE-RADAU')

NotImplementedError: Variance estimator requires spectral data in model as model.D[ti,lj]

In [ ]:
sigmas = {'A':1e-10,'B':1e-11,'C':1e-8}
p_estimator = ParameterEstimator(opt_model)
p_estimator.apply_discretization('dae.collocation',nfe=100,ncp=3,scheme='LAGRANGE-RADAU')

In [ ]:
options = dict()
#options['nlp_scaling_method'] = 'gradient-based'
#options['bound_relax_factor'] = 0
# finally we run the optimization
results_pyomo = p_estimator.run_opt('k_aug',
                                    variances=sigmas,
                                  tee=True,
                                  solver_opts = options,
                                  covariance=True)

In [ ]:
print("The estimated parameters are:")
for k,v in six.iteritems(results_pyomo.P):
    print(k, v)